In [2]:
# 导入库
import pandas as pd
import numpy as np
from tqdm import tqdm
# 用于读取和显示图像
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
%matplotlib inline
# 用于创建验证集
from sklearn.model_selection import train_test_split
# 用于评估模型
from sklearn.metrics import accuracy_score
# PyTorch库和模块
import torch
from torch.autograd import Variable
from torch.nn import Linear,ReLU,CrossEntropyLoss,Sequential,Conv2d,MaxPool2d,Module,Softmax,BatchNorm2d,Dropout
from torch.optim import Adam,SGD
# 预训练模型
from torchvision import models

AttributeError: type object 'skimage.transform._hough_transform.array' has no attribute '__reduce_cython__'